<a href="https://colab.research.google.com/github/manan-garg/FAISS_RecSys_Master/blob/main/FAISS_RecSys_Master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### To understand FAISS (Facebook AI Similarity Search) and leverage it for efficient vector similarity search in order to develop a recommendation system.

# 1. Install and Import relevant Libraries

In [ ]:
# Installing relevant libraries
!pip install sentence_transformers
!pip install datasets
!pip install faiss-gpu
!pip install faiss-cpu

# Importing relevant libraries
from datasets import load_dataset
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import warnings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.5 MB/s eta 0:00:00


# 2. Set Options

In [ ]:
warnings.simplefilter('ignore')
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:.2f}'.format

# 3. Load Dataset

Here we are using a small dataset of book titles and their descriptions for our use-case.



In [ ]:
dataset = load_dataset('Skelebor/book_titles_and_descriptions_en_clean', split='test')
df = pd.DataFrame(dataset)
df.head()

Generating train split:   0%|          | 0/1032335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/57352 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/57352 [00:00<?, ? examples/s]

,title,description
0,The Baby of Their Dreams,"Barcelona, baby...bride?\nSeven years ago ER d..."
1,"Air Gear, Vol. 8 (Air Gear, #8)",Behemoth has already taken out part of Ikki's ...
2,Walking Over Eggshells,Walking Over Eggshells is an autobiography tha...
3,"Charmed (Fairy Tale Reform School, #2)",Charmed is the exciting sequel to the wildly p...
4,"Blown Away (Unconventional in Atlanta, #2)",Sometimes love finds you before you think you'...


# 4. Basic Metrics

In [ ]:
# Shape of the dataset -
df.shape

(57352, 2)

In [ ]:
# Removing duplicate rows
df.drop_duplicates(inplace=True)

In [ ]:
# Checking number of Nulls
df.isnull().sum(axis=1).sum()

0

# 5. Embeddings

We'll gather all the descriptions and create sentence embeddings for each one, which can subsequently be stored in FAISS.

In [ ]:
# Creating list of sentences
sentences = df['description'].tolist()

Next, we'll proceed with constructing the sentence embeddings.

In [ ]:
# Creating sentence embeddings
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(sentences)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 6. Vector Similarity Search Implementation

FAISS, short for Facebook AI Similarity Search, is an open-source library created by Facebook AI Research (FAIR) to facilitate efficient similarity search and clustering of high-dimensional vectors. It is designed to excel in situations with extensive datasets and high-dimensional feature vectors, which are frequently encountered in tasks like image and text retrieval, recommendation systems, and natural language processing.

We configure the dimensionality of our FAISS database according to these vector embeddings.

In [ ]:
# Vector dimensionality
vec_dim = sentence_embeddings.shape[1]
vec_dim

768

In [ ]:
# Setting number of nearest neighbours
k = 4

In [ ]:
# Creating Search Query embedding
query = model.encode(["Aliens and Spaceships"])

## 6.1 Flat L2 Index

We begin by initializing the flat L2 distance index 'IndexFlatL2', where we only need to specify the vector dimensionality. In this case, the dimensionality is vec_dim = 768, which corresponds to the output embeddings size of the sentence-BERT model.

In [ ]:
# Creating index
index = faiss.IndexFlatL2(vec_dim)
index.add(sentence_embeddings)

Once we are satisfied that our index is prepared, we proceed to add new vectors using the add method.

Next, we'll locate the nearest query embeddings to our search query embedding based on the specified value of 'k'.



In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[21656 44760 10863 35757]]
CPU times: user 18.8 ms, sys: 0 ns, total: 18.8 ms
Wall time: 19.6 ms


After selecting the descriptions that closely match the search query, we print the respective titles of the books associated with those descriptions.

In [ ]:
# Printing nearest neighbours for book titles
[f'{i}: {df.iloc[i].title}' for i in I[0]]

['21656: Aliens; Extraterresterial Tales of Terror',
 '44760: Edward Built a Rocketship',
 '10863: Across the Sea of Suns (Galactic Center, #2)',
 '35757: Look Inside Space']

Indeed, it appears we have found some promising matches. All the recommendations are related to either space, aliens, or both. Adjusting the number of recommendations can be achieved by updating the value of 'k'.

## 6.2 Adding Partitioning to the Index

FAISS indeed provides the option to enhance our search efficiency through various methods, with partitioning the index into [Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram) being a popular approach. With this method, we take our query vector, determine the cell it belongs to, and then utilize our 'IndexFlatL2' to search among the query vector and all indexed vectors within that cell. Additionally, we can include vectors from other nearby cells if needed.

In [ ]:
# Defining number of partitions
nlist = 200

In this updated setup, we've introduced a new parameter called 'nlist'. This parameter allows us to specify the number of partitions we want our index to have.

In [ ]:
# Creating index
quantizer = faiss.IndexFlatL2(vec_dim)
index = faiss.IndexIVFFlat(quantizer, vec_dim, nlist)

We initialize our new partitioned index by incorporating our previous 'IndexFlatL2' operation as a quantization step, which serves as another stage in the search process. Then, we feed this into the new 'IndexIVFFlat' operation.

In [ ]:
# Training index
index.train(sentence_embeddings)

With the inclusion of partitioning using 'IndexIVFFlat', training becomes necessary as grouping and transformation are involved in building this index. Therefore, before adding any data to the index, we need to train it on our dataset. This step is essential so that the index can appropriately group each vector.

In [ ]:
# Adding sentence embeddings
index.add(sentence_embeddings)

After training our index, we proceed to add our data in the same manner as before. We can then conduct our search again using the same indexed sentence embeddings and the same 'query'.

In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[21656 44760 35757 10947]]
CPU times: user 2.34 ms, sys: 0 ns, total: 2.34 ms
Wall time: 2.08 ms


In [ ]:
# Printing nearest neighbours for book titles
[f'{i}: {df.iloc[i].title}' for i in I[0]]

['21656: Aliens; Extraterresterial Tales of Terror',
 '44760: Edward Built a Rocketship',
 '35757: Look Inside Space',
 '10947: Starman, Vol. 7: A Starry Knight']

We can also adjust the number of nearby cells to search using the parameter 'nprobe'.

In [ ]:
# Increasing number of nearby cells to search
index.nprobe = 20

In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[21656 44760 10863 35757]]
CPU times: user 2.99 ms, sys: 0 ns, total: 2.99 ms
Wall time: 2.27 ms


In [ ]:
# Printing nearest neighbours for book titles
[f'{i}: {df.iloc[i].title}' for i in I[0]]

['21656: Aliens; Extraterresterial Tales of Terror',
 '44760: Edward Built a Rocketship',
 '10863: Across the Sea of Suns (Galactic Center, #2)',
 '35757: Look Inside Space']

Increasing the value of 'nprobe' can enhance the accuracy of our search, but at the expense of time. In our previous search using only 'IndexFlatL2', which was exhaustive (comparing every single vector), we achieved perfect accuracy in identifying the closest matches. By reducing the 'nprobe' value, we limit the scope of our search. While we obtained perfect results matching our previous 'IndexFlatL2' only results, if we find that the results are not closely matching, we can increase 'nprobe' to improve accuracy, albeit at the cost of increased time. It's important to note that the time taken for the search can vary with each run as well.

## 6.3 Quantization

Now that we have significantly reduced the search time, the next step is to address the issue of storage space consumption when storing full vectors, especially in large datasets.

Fortunately, FAISS offers a solution by providing the ability to compress vectors using transformations based on [Product Quantization (PQ)](https://mccormickml.com/2017/10/13/product-quantizer-tutorial-part-1/). But what exactly is PQ?

PQ can be seen as an additional approximation step, akin to our use of 'IVF', which allowed us to approximate by reducing the scope of our search. However, PQ differs slightly as it approximates the distance (or similarity) calculation instead.

PQ achieves this by compressing the vectors themselves, which involves several steps:

1. We partition each original vector into several subvectors.
2. For each set of subvectors, we conduct a clustering operation, generating numerous centroids for each set of subvectors.
3. Within our vector of subvectors, we replace each subvector with the ID of its nearest centroid.

This compression process helps alleviate the storage space issue associated with storing full vectors.

In [ ]:
# Number of centroid IDs
num_cent_ids = 8

# Number of bits in each centroid
cent_bits = 8

In [ ]:
# Keeping the same L2 distance flat index
quantizer = faiss.IndexFlatL2(vec_dim)
index = faiss.IndexIVFPQ(quantizer, vec_dim, nlist, num_cent_ids, cent_bits)

Once more, we'll have to train the index and add our vectors before proceeding further.

In [ ]:
# Training index
index.train(sentence_embeddings)

# Adding sentence embeddings
index.add(sentence_embeddings)

Let's compare it with our previous index, which didn't utilize Product Quantization (PQ) and had a 'nprobe' value of 20.

In [ ]:
# Defining number of nearby cells to search
index.nprobe = 20

In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[21656 28903 42857 25568]]
CPU times: user 1.52 ms, sys: 0 ns, total: 1.52 ms
Wall time: 1.54 ms


By incorporating Product Quantization (PQ), we have successfully reduced our search time. Although the difference may be small on a dataset of this size, it can make a significant impact when scaled to larger datasets.

It's important to note that with these speed optimization techniques (IVF and PQ), there might be slight differences in the results returned. However, upon printing out these results, we'll observe that each item is still a relevant match.

In [ ]:
# Printing nearest neighbours for book titles
[f'{i}: {df.iloc[i].title}' for i in I[0]]

['21656: Aliens; Extraterresterial Tales of Terror',
 '28903: One Step from Earth',
 '42857: ABCs from Space: A Discovered Alphabet',
 "25568: The Kingfisher Young People's Book of Space"]

In conclusion, despite not achieving perfect accuracy, the close results obtained through approximation techniques like 'IVF' and 'PQ' provide substantial speed enhancements.